# Rasterization Script

## Overview
This notebook contains a Python script for converting vector datasets (e.g., shapefiles) into raster datasets using GDAL. The script automates the rasterization process by allowing users to specify parameters such as resolution, NoData value, and output raster data type. It also includes utility functions for folder management and batch processing of vector files.

### Parameters
- **`shapefile_path` (str)**: Path to the input vector file (e.g., a shapefile).
- **`resolution` (int or float)**: The pixel size (in units of the coordinate reference system) for the output raster.
- **`column_name` (str)**: The attribute column with values in the vector file to use for rasterization.
- **`nodata_value` (int or float)**: The value assigned to NoData pixels in the raster.
- **`data_type` (int)**: The GDAL data type for the raster (e.g., `gdal.GDT_Float32` or `gdal.GDT_UInt16`).
- **`output_path` (str)**: Path to save the output raster.
- **`cols` (int, optional)**: Number of columns for the raster (calculated automatically if not provided).
- **`rows` (int, optional)**: Number of rows for the raster (calculated automatically if not provided).

### Notes
- Includes a utility function to list vector files from a directory.
- Handles batch processing of shapefiles in a folder, making it ideal for large-scale rasterization projects.
- Uses GDAL for efficient rasterization and allows fine-tuning of raster properties.

## Author
- **Rubén Crespo Ceballos**


In [ ]:
from osgeo import gdal, ogr
import os

In [ ]:
def create_folder_if_not_exists(folder_path):
    """
    Create a folder if it doesn't exist.

    Parameters:
    folder_path (str): The path of the folder to be created.
    """
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
        print(f"Folder created at: {folder_path}")
    else:
        print(f"Folder already exists at: {folder_path}")

def get_vector_file_list(path):
    """
    Get a list of the vector files inside the folder
    Parameters:
    - path (str): path of the folder with the resources.

    Returns:
    - File_list (list). list of the resources.
    """
    File_list = [] #f for f in os.listdir(path) if os.isfile(mypath,f)
    for file in os.listdir(path):
        # "anat" is just to get here necessary ones
        if file.endswith(".shp"):
            if file not in File_list:
                File_list.append(os.path.join(path,file))
        else:
            pass
    return File_list

def gdal_rasterize_from_shapefile(shapefile_path, resolution, column_name, nodata_value, data_type, output_path, cols=None, rows=None):
    """
    Rasterizes a GeoDataFrame using GDAL directly.

    Parameters:
    - shapefile_path (string): path of the vector file.
    - resolution (int or float): Resolution of the raster (pixel size).
    - nodata_value: The value to use for no-data pixels.
    - data_type: Data type for the output raster (e.g., gdal.GDT_Float32).
    - output_path (str): Path to save the output raster.
    - cols (int, optional): Number of columns in the output raster.
    - rows (int, optional): Number of rows in the output raster.

    Returns:
    - None. The function writes the raster to the specified output path.
    """

    # Open the Shapefile using OGR
    shapefile = ogr.Open(shapefile_path)
    layer = shapefile.GetLayer()

    # Get the bounds of the Shapefile (same as GeoDataFrame's total_bounds)
    xmin, xmax, ymin, ymax = layer.GetExtent()

    # If cols and rows are not provided, calculate them based on resolution
    if cols is None or rows is None:
        cols = int((xmax - xmin) / resolution)
        rows = int((ymax - ymin) / resolution)

    # Create a new raster dataset
    raster_ds = gdal.GetDriverByName('GTiff').Create(
        output_path, cols, rows, 1, data_type,
        options=['COMPRESS=DEFLATE', 'TILED=YES']
    )

    # Set the geotransform (affine transform for the raster)
    geotransform = (xmin, resolution, 0, ymax, 0, -resolution)
    raster_ds.SetGeoTransform(geotransform)

    # Set the CRS (coordinate reference system) from the Shapefile
    srs = layer.GetSpatialRef()
    if srs:
        raster_ds.SetProjection(srs.ExportToWkt())

    # Create the raster band and set no-data value
    band = raster_ds.GetRasterBand(1)
    band.SetNoDataValue(nodata_value)

    # Rasterize the shapefile
    gdal.RasterizeLayer(
        raster_ds,  # Output raster dataset
        [1],        # Raster band to write to
        layer,      # Input OGR layer to rasterize
        options=[f'ATTRIBUTE={column_name}', 'ALL_TOUCHED=TRUE']
    )

    # Flush and close the raster dataset
    band.FlushCache()
    raster_ds = None  # Close the file and save

    # Close the shapefile
    shapefile = None

    print(f"Rasterization complete: {output_path}")

In [ ]:
"""Specify all the inputs"""
#"Y:\z_resources\im-nca-senegal\v2_shp_occsol_anat\23-12-22\shp_occsol_anat"
input_path = r"Y:\z_resources\ruben\ladncover_test" # Put the year a the end of the files
output_path = input_path + r"\output_files"

create_folder_if_not_exists(output_path)

# Specify the column of the vector file
column_name = 'column_name'
# Define the resolution of your raster.
resolution = 30  # in meters
# Define the nodata value of your raster.
nodata_value = 0

# Define the data type of the raster.
data_type = gdal.GDT_UInt16
"""
gdal.GDT_Byte,
gdal.GDT_Int16,
gdal.GDT_UInt16,
gdal.GDT_Int32,
gdal.GDT_UInt32,
gdal.GDT_Float32,
gdal.GDT_Float64
"""

# Define the rows and columns for the rasterization. Assign None if you don't want to specify it
rows = 9999 # None
columns = 9999 # None

Folder already exists at: Y:\z_resources\ruben\ladncover_test\output_files


In [ ]:
"""Convert vectors into rasters""" # Direct way
vector_file_list = get_vector_file_list(input_path)
for file in vector_file_list[:]:
    output_path_file = os.path.join(output_path, os.path.basename(file).replace(".shp", ".tif"))
    print(output_path_file)
    gdal_rasterize_from_shapefile(file, resolution, column_name, nodata_value, data_type, output_path_file, cols=None, rows=None)
